# <span style="color: blue;">ให้กรอกรหัสนิสิต และชื่อ-นามสกุลของสมาชิกในทีม</span>
# <span style="color: blue;">(ห้ามมีสมาชิกเกิน 2 คน)</span>
## รหัสนิสิต ชื่อ-นามสกุล

## Image Classification: Comparing No Augmentation, Data Augmentation, and Pretrained Models

This exercise investigates image classification on the CIFAR-100 dataset by comparing three different training strategies: a baseline model trained without any augmentation, a model trained with data augmentation techniques to improve generalization, and a model leveraging transfer learning with pretrained weights. The goal is to evaluate how each approach affects classification performance, focusing on accuracy, loss behavior, and robustness.

In [ ]:
import os

# must come *before* any Tensorflow import
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import numpy as np
import os, random, tensorflow as tf
import requests, pickle

def set_seed(seed_value=12345):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

## Testing ImageDataGenerator

In [ ]:
import gdown, requests
from PIL import Image
import numpy as np

gdown.download("https://drive.google.com/uc?export=download&id=1xUwP3DVt690D2NpgDdoNaNE37uoa0a3V", 
               "coffee_mug.jpg", quiet=True)
image = Image.open("coffee_mug.jpg")
image = np.expand_dims(image, axis=0).astype('float32')/255
plt.imshow(image[0])

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

image_generator = datagen.flow(image)

In [ ]:
plt.imshow(next(image_generator)[0])

## Loading Dataset

In [ ]:
url = "https://drive.google.com/uc?id=1KiNBzO6Rglrs96Ar92sVjYckAvm2A0Fi"
(_X, _y) = pickle.loads(requests.get(url).content)

class_names = ['bottle', 'bowl', 'can', 'cup', 'plate']

## <span style="color:orange"> เกณฑ์การให้คะแนน (คะแนนเต็ม 100 คะแนน)</span>
* ### <span style="color:darkcyan"> หากสามารถทำตามข้อกำหนดได้ครบและสมบูรณ์ทุกข้อ พร้อมอธิบายและเปรียบเทียบประสิทธิภาพของโมเดลทั้งหมดได้อย่างชัดเจน ได้รับคะแนนเต็ม 100 คะแนน</span>
* ### <span style="color:darkcyan"> หากมีข้อบกพร่องน้อยหรืออธิบายได้ดีพอประมาณ จะได้รับ 70 คะแนน</span>
* ### <span style="color:darkcyan"> หากมีข้อบกพร่องหรือโค้ดทำงานไม่ถูกต้อง จะได้รับ 40 คะแนน</span>
* ### <span style="color:darkcyan"> หากมีข้อบกพร่องอย่างมาก โค้ดรันไม่ได้ หรือไม่ได้เขียนโค้ดส่งมา จะได้รับศูนย์คะแนน</span>
* ### <span style="color:darkcyan"> หากโจทย์ไม่ได้กำหนดให้ทำ </span><span style="color:red">ห้ามใช้ Early Stopping, ห้ามใช้ Best Weights, ห้ามใช้ Data Augmentation และห้ามใช้โมเดล Pretrained ในการเทรนโมเดล</span>

## <span style="color:green"> 1. จากชุดข้อมูลที่เตรียมไว้ให้ซึ่งอยู่ในตัวแปร _X และ _y ให้แบ่งข้อมูลออกเป็นชุดข้อมูล train, val, และ test ด้วยอัตราส่วน 70:10:20 และให้ทุกชุดข้อมูลมีจำนวนข้อมูลของแต่ละคลาสเท่ากัน หลังจากที่แบ่งชุดข้อมูลเสร็จ ให้แสดงจำนวนข้อมูลของแต่ละคลาสในทุกชุดข้อมูล</span>

## <span style="color:green"> 2. ในแต่ละชุดข้อมูล ให้แสดงภาพพร้อมชื่อคลาสออกมาเป็นจำนวน 12 ภาพ</span>

### Normalize and One-Hot Encode Datasets โดยปรับโค้ดข้างล่างให้ตรงกับตัวแปรที่ใช้

In [ ]:
# n_classes = len(class_names)

# X_train = X_train.astype('float32') / 255
# X_val = X_val.astype('float32') / 255
# X_test = X_test.astype('float32') / 255

# y_train = keras.utils.to_categorical(y_train, n_classes)
# y_val = keras.utils.to_categorical(y_val, n_classes)
# y_test = keras.utils.to_categorical(y_test, n_classes)

## <span style="color:green"> 3. ให้สร้าง เทรน และประเมินผลโมเดล CNN โดยให้ตั้งชื่อตัวแปรโมเดลว่า model_1 ซึ่งในการเทรนโมเดลนี้ห้ามใช้ Data Augmentation</span>

## <span style="color:green"> 4. (20 คะแนน) ให้นำโครงสร้างโมเดล model_1 มาสร้างโมเดลที่มีชื่อตัวแปรโมเดลว่า model_2 โดยห้ามปรับเปลี่ยนโครงสร้างโมเดล แต่ให้เทรนโมเดลโดยใช้ Data Augmentation และใช้จำนวน epoch เท่ากับโมเดลก่อนหน้า แล้วจึงประเมินผลโมเดล</span>

## <span style="color:green"> 5. ให้เรียกใช้ฟังก์ชันเพื่อดาวน์โหลดโมเดล Pretrained และทำ fine-tuning เพื่อแสดงความเข้าใจในเรื่อง Transfer Learning โดยให้ตั้งชื่อตัวแปรโมเดลนี้ว่า model_3 และเทรนโมเดลเพิ่มด้วยการใช้ Data Augmentation และใช้จำนวน epoch ไม่เกิน 20% ของค่าที่ใช้ในการเทรน model_2</span>